# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2920225370>
├── 'a' --> tensor([[-0.0033,  0.1689,  0.1574],
│                   [-0.5955, -2.2964,  0.9161]])
└── 'x' --> <FastTreeValue 0x7f2920225ee0>
    └── 'c' --> tensor([[ 1.9237, -0.6604,  0.4551, -0.9099],
                        [ 0.4770, -0.6154,  1.4621, -1.2204],
                        [ 0.7539,  0.0613,  1.5647,  0.4247]])

In [4]:
t.a

tensor([[-0.0033,  0.1689,  0.1574],
        [-0.5955, -2.2964,  0.9161]])

In [5]:
%timeit t.a

65.6 ns ± 0.016 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2920225370>
├── 'a' --> tensor([[-1.6752,  0.6100, -0.0267],
│                   [ 0.1356, -1.7119,  0.1608]])
└── 'x' --> <FastTreeValue 0x7f2920225ee0>
    └── 'c' --> tensor([[ 1.9237, -0.6604,  0.4551, -0.9099],
                        [ 0.4770, -0.6154,  1.4621, -1.2204],
                        [ 0.7539,  0.0613,  1.5647,  0.4247]])

In [7]:
%timeit t.a = new_value

72.8 ns ± 0.0354 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0033,  0.1689,  0.1574],
               [-0.5955, -2.2964,  0.9161]]),
    x: Batch(
           c: tensor([[ 1.9237, -0.6604,  0.4551, -0.9099],
                      [ 0.4770, -0.6154,  1.4621, -1.2204],
                      [ 0.7539,  0.0613,  1.5647,  0.4247]]),
       ),
)

In [10]:
b.a

tensor([[-0.0033,  0.1689,  0.1574],
        [-0.5955, -2.2964,  0.9161]])

In [11]:
%timeit b.a

58.7 ns ± 0.0488 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.2380, -0.2040, -0.7764],
               [ 0.6612,  0.1167, -0.8407]]),
    x: Batch(
           c: tensor([[ 1.9237, -0.6604,  0.4551, -0.9099],
                      [ 0.4770, -0.6154,  1.4621, -1.2204],
                      [ 0.7539,  0.0613,  1.5647,  0.4247]]),
       ),
)

In [13]:
%timeit b.a = new_value

495 ns ± 0.0925 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

858 ns ± 0.221 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.6 µs ± 16.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 900 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 550 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f285d63cdc0>
├── 'a' --> tensor([[[-0.0033,  0.1689,  0.1574],
│                    [-0.5955, -2.2964,  0.9161]],
│           
│                   [[-0.0033,  0.1689,  0.1574],
│                    [-0.5955, -2.2964,  0.9161]],
│           
│                   [[-0.0033,  0.1689,  0.1574],
│                    [-0.5955, -2.2964,  0.9161]],
│           
│                   [[-0.0033,  0.1689,  0.1574],
│                    [-0.5955, -2.2964,  0.9161]],
│           
│                   [[-0.0033,  0.1689,  0.1574],
│                    [-0.5955, -2.2964,  0.9161]],
│           
│                   [[-0.0033,  0.1689,  0.1574],
│                    [-0.5955, -2.2964,  0.9161]],
│           
│                   [[-0.0033,  0.1689,  0.1574],
│                    [-0.5955, -2.2964,  0.9161]],
│           
│                   [[-0.0033,  0.1689,  0.1574],
│                    [-0.5955, -2.2964,  0.9161]]])
└── 'x' --> <FastTreeValue 0x7f285d621160>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 62.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f285552e460>
├── 'a' --> tensor([[-0.0033,  0.1689,  0.1574],
│                   [-0.5955, -2.2964,  0.9161],
│                   [-0.0033,  0.1689,  0.1574],
│                   [-0.5955, -2.2964,  0.9161],
│                   [-0.0033,  0.1689,  0.1574],
│                   [-0.5955, -2.2964,  0.9161],
│                   [-0.0033,  0.1689,  0.1574],
│                   [-0.5955, -2.2964,  0.9161],
│                   [-0.0033,  0.1689,  0.1574],
│                   [-0.5955, -2.2964,  0.9161],
│                   [-0.0033,  0.1689,  0.1574],
│                   [-0.5955, -2.2964,  0.9161],
│                   [-0.0033,  0.1689,  0.1574],
│                   [-0.5955, -2.2964,  0.9161],
│                   [-0.0033,  0.1689,  0.1574],
│                   [-0.5955, -2.2964,  0.9161]])
└── 'x' --> <FastTreeValue 0x7f285552e250>
    └── 'c' --> tensor([[ 1.9237, -0.6604,  0.4551, -0.9099],
                        [ 0.4770, -0.6154,  1.4621, -1.2204],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 29.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69 µs ± 67.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.9237, -0.6604,  0.4551, -0.9099],
                       [ 0.4770, -0.6154,  1.4621, -1.2204],
                       [ 0.7539,  0.0613,  1.5647,  0.4247]],
              
                      [[ 1.9237, -0.6604,  0.4551, -0.9099],
                       [ 0.4770, -0.6154,  1.4621, -1.2204],
                       [ 0.7539,  0.0613,  1.5647,  0.4247]],
              
                      [[ 1.9237, -0.6604,  0.4551, -0.9099],
                       [ 0.4770, -0.6154,  1.4621, -1.2204],
                       [ 0.7539,  0.0613,  1.5647,  0.4247]],
              
                      [[ 1.9237, -0.6604,  0.4551, -0.9099],
                       [ 0.4770, -0.6154,  1.4621, -1.2204],
                       [ 0.7539,  0.0613,  1.5647,  0.4247]],
              
                      [[ 1.9237, -0.6604,  0.4551, -0.9099],
                       [ 0.4770, -0.6154,  1.4621, -1.2204],
                       [ 0.7539,  0.0613,  1.5647,  0.4247]],

In [26]:
%timeit Batch.stack(batches)

79.3 µs ± 335 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.9237, -0.6604,  0.4551, -0.9099],
                      [ 0.4770, -0.6154,  1.4621, -1.2204],
                      [ 0.7539,  0.0613,  1.5647,  0.4247],
                      [ 1.9237, -0.6604,  0.4551, -0.9099],
                      [ 0.4770, -0.6154,  1.4621, -1.2204],
                      [ 0.7539,  0.0613,  1.5647,  0.4247],
                      [ 1.9237, -0.6604,  0.4551, -0.9099],
                      [ 0.4770, -0.6154,  1.4621, -1.2204],
                      [ 0.7539,  0.0613,  1.5647,  0.4247],
                      [ 1.9237, -0.6604,  0.4551, -0.9099],
                      [ 0.4770, -0.6154,  1.4621, -1.2204],
                      [ 0.7539,  0.0613,  1.5647,  0.4247],
                      [ 1.9237, -0.6604,  0.4551, -0.9099],
                      [ 0.4770, -0.6154,  1.4621, -1.2204],
                      [ 0.7539,  0.0613,  1.5647,  0.4247],
                      [ 1.9237, -0.6604,  0.4551, -0.9099],
                   

In [28]:
%timeit Batch.cat(batches)

146 µs ± 266 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

344 µs ± 1.66 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
